In [1]:
#Standard Imports
import os
import sys
import pandas as pd
# from collections import defaultdict
#import timeit
from docplex.cp.model import CpoModel

In [2]:
#Custom Imports
sys.path.append('../')
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput
from src.cp_model import CPModel
from src.music_functions import *
from src.evaluate import *
from src.midi_processing import *

In [3]:
# Importing Musical Corpus
musical_work_df = pd.read_csv("../data/sample_input.csv")
musical_corpus = []
for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [4]:
# Importing Chord Vocabulary
chord_df_major = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
chord_df_minor = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab_major, chord_vocab_minor = [], []
for index, name, note_intervals in chord_df_major.itertuples():
    chord_vocab_major.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))
for index, name, note_intervals in chord_df_minor.itertuples():
    chord_vocab_minor.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))

In [5]:
# Defining penalties for chord progression
penalties_chord_progression_major = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
penalties_chord_progression_minor = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)
penalties_chord_progression_major = dict(penalties_chord_progression_major.stack())
penalties_chord_progression_minor = dict(penalties_chord_progression_minor.stack())

In [6]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights_temp.csv")

# Defining dictionary of weights for each soft constraint option:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
assert sum(v for v in soft_constraint_w_weights.values() if v > 0) == 100

In [7]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'first inversion', 'chord distribution']

In [8]:
# Defining which hard constraints to use
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords',
                                     'voice crossing', 'parallel movement',
                                     'chord spacing', 'incomplete chord'] else False for x in hard_constraint_options}

In [9]:
# Model
for musical_work in [musical_corpus[-1]]:

    print('*'*20)
    print('Work: {}'.format(musical_work.title))
    print('*'*20)
    
    if musical_work.tonality == 'major':
        penalties_chord_progression = penalties_chord_progression_major
        chord_vocab = chord_vocab_major
    else:
        penalties_chord_progression = penalties_chord_progression_minor
        chord_vocab = chord_vocab_minor        
    
    #Defining Model
    cp_model = CPModel('Completed', musical_work, chord_vocab, penalties_chord_progression,
                       hard_constraints,
                       soft_constraint_w_weights)
    
    #Solving Model
    solution = cp_model.solve(log_output = True, TimeLimit = 600, LogVerbosity = 'Verbose')
    result = cp_model.get_solution()
    cp_model.export_midi()
    cp_model.export_csv()

********************
Work: Ach bleib' bei unsm Herr Jesu Christ (part)
********************
(127,)
-------------------------------------------------------------------------------
Model constraints: 66295, variables: integer: 1460, interval: 0, sequence: 0
Solve status: Feasible
Search status: SearchStopped, stop cause: SearchStoppedByLimit
Solve time: 602.68 sec
-------------------------------------------------------------------------------
Objective values: (127,), bounds: (12,), gaps: (0.905512,)
Chords_0=0
Chords_1=0
Chords_2=7
Chords_3=0
Chords_4=6
Chords_5=7
Chords_6=0
Chords_7=6
Chords_8=0
Chords_9=7
Chords_10=0
Chords_11=6
Chords_12=7
Chords_13=0
Chords_14=0
Chords_15=7
Chords_16=7
Chords_17=0
Chords_18=6
Chords_19=0
Notes_0=37
Notes_1=37
Notes_2=40
Notes_3=37
Notes_4=33
Notes_5=35
Notes_6=37
Notes_7=38
Notes_8=37
Notes_9=35
Notes_10=33
Notes_11=33
Notes_12=35
Notes_13=37
Notes_14=37
Notes_15=35
Notes_16=35
Notes_17=33
Notes_18=33
Notes_19=33
Notes_20=33
Notes_21=33
Notes_22=35


In [10]:
list_x = cp_model.sol_var['Notes']
list_c = []
for chord_name in cp_model.sol_var['Chords']:
    exists = False
    for chord in chord_vocab:
        if chord.name == chord_name:
            exists = True
            list_c.append(chord.index)
            break
    if not exists:
        list_c.append(-1)

In [12]:
total_cp_cost = evaluate_cost(list_x, list_c, cp_model.K, cp_model.musical_input.tonality,
                     cp_model.musical_input.meter, cp_model.musical_input.first_on_beat, 'D',
                     chord_vocab,
                     penalties_chord_progression,
                     hard_constraints, 1000,
                     soft_constraint_w_weights)
print(sum(total_cp_cost.values()))

107.33333333333334
